# Evaluate the performance of model using Backtest Pipeline
In this second example notebook, the performance of the model is analysed using ``train_model_and_forecast_back_test``. First, the prediction job is defined, where the properties of the training and forecasting are specified. Thereafter, the backtest is performed using the prediction job and input data (can be found in the 'data' folder). Thereafter, the results are analysed. 

In [ ]:
import pandas as pd
from openstef.pipeline.train_create_forecast_backtest import train_model_and_forecast_back_test
from openstef.metrics.figure import plot_feature_importance
from openstef.data_classes.model_specifications import ModelSpecificationDataClass
from openstef.data_classes.prediction_job import PredictionJobDataClass # TODO, import from openstef when availavle
from openstef.plotting.load_forecast_plotter import LoadForecastPlotter

# Set plotly as the default pandas plotting backend
pd.options.plotting.backend = 'plotly'
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

## Prepare for training & backtest
Before a model can be trained, the specifications and data need to be defined. The specification of the model are defined in the prediction job (pj), where for example the machine learning model, latitude, longtide and forecast horizon are specified. Furthermore, the data has to be retrieved from the csv file containing both load, weather and energy market data. 

In [ ]:
# Define properties of training/prediction. We call this a 'prediction_job' 
pj=PredictionJobDataClass(id=287,
        model='xgb',
        quantiles=[0.10,0.30,0.50,0.70,0.90],
        horizon_minutes=48*60,
        resolution_minutes=15,
        lat = 1, 
        lon = 1, 
        train_components=False,
        name='TestPrediction',
        model_type_group=None, 
        hyper_params={}, 
        feature_names=None, 
        forecast_type="demand", 
                  )

modelspecs = ModelSpecificationDataClass(id=pj['id'])

# Load input data
input_data = pd.read_csv('data/get_model_input_pid_287.csv', index_col='index', parse_dates=True)


## Perform backtest
Below, the backtest is performed using ``train_model_and_forecast_back_test``, which not only outputs the forecast but also the model, train data, validation data and test data. The availability of both the forecast and realised values, enables you to evaluate the results of the model.

One of the variables in the ``train_model_and_forecast_back_test`` are the ``training_horizons``. This entails how far into the future, the model has to predict. Thus, a value of 0.25 means predicting 15 minutes into the future, where as 47.0 entails predicting 47 hours ahead.

In [ ]:
# Perform the backtest
n_folds = 2

forecast, model, train_data, validation_data, test_data = train_model_and_forecast_back_test(
    pj,
    modelspecs = modelspecs,
    input_data = input_data,
    training_horizons=[0.25, 47.0],
    n_folds=n_folds,
 )

# If n_folds>1, model is a list of models. In that case, only use the first model
if n_folds>1:
    model=model[0]

## Evaluate results
Below, the results of the backtest will be evaluated by means of visualisation (plots) and metrics. 

In [ ]:
for horizon in set(forecast.horizon):
    data = forecast.loc[forecast.horizon==horizon]

    fig = LoadForecastPlotter().plot(
      realized=data["realised"],
      forecast=data["forecast"],
      quantiles=data.filter(regex="quantile")
    )
    fig.update_layout(dict(title=f'Horizon {horizon}'))
    fig.show()

Evaluate the error of the forecast by subtracting the realised values by the forecasted values. The visualisation can help to analyse the errors.

In [ ]:
forecast['err']=forecast['realised']-forecast['forecast']
forecast['err'].plot()

The mean absolute error (mea) gives insight into the scale of the errors. 

In [ ]:
mea = forecast.pivot_table(index='horizon', values=['err'], aggfunc=lambda x: x.abs().mean())
mea.index=mea.index.astype(str)
fig = mea.plot(kind='bar')
fig.update_layout(dict(title='MAE',
                      xaxis=dict(title='horizon'),
                      yaxis=dict(title='MAE [MW]')))

Lastly, it is of interest too look into the importance of the features the model has used to make the forecast. The larger the block in this plot, the higher the importance of the feature for the forecast.

In [ ]:
plot_feature_importance(model.feature_importance_dataframe)